# Tutorial 1: Calculando o índice do ENSO com o `xarray`

**Autor:** Elisa Passos

## Importando as bibliotecas

In [ ]:
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.patches import Polygon
import matplotlib.dates as mdates
import numpy as np
from datetime import datetime
from pythia_datasets import DATASETS
from dateutil.relativedelta import relativedelta
import pandas as pd
import xarray as xr
from noaa_ftp import NOAA
import cftime

import cartopy.crs as ccrs
import cartopy.io.shapereader as shapereader

import geoviews as gv
import geoviews.feature as gf

# Objetivos do Tutorial

Neste caderno (adaptado do [Projeto Pythia](https://foundations.projectpythia.org/core/xarray/enso-xarray.html)), você irá praticar o uso de várias ferramentas para examinar a temperatura da superfície do mar (SST) e explorar variações no sistema climático que ocorrem durante os eventos El Niño e La Niña. Você aprenderá a:

1. Carregar dados de Temperatura da Superfície do Mar do produto OISST
2. Mascarar dados usando `.where()`
3. Calcular climatologias e anomalias usando `.groupby()`
4. Usar `.rolling()` para calcular a média móvel
5. Calcular, normalizar e plotar o Índice Oceânico Niño

Após completar as tarefas acima, você deverá ser capaz de plotar o [Índice Oceânico Niño](https://climatedataguide.ucar.edu/climate-data/nino-sst-indices-nino-12-3-34-4-oni-and-tni) que se assemelha à figura abaixo. As regiões vermelha e azul correspondem às fases de El Niño e La Niña, respectivamente.

![Gráfico do índice ONI do Guia de Dados Climáticos da NCAR](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

Crédito: [NCAR](https://climatedataguide.ucar.edu/sites/default/files/styles/extra_large/public/2022-03/indices_oni_2_2_lg.png)

Crédito do Pythia: Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Versão v2023.05.01) https://zenodo.org/record/8065851

# Seção 1: Introdução à Oscilação Sul do El Niño (ENSO)

A região geográfica compreendida entre **5ºN-5ºS e 190ºE-240ºE (ou 170ºW-120ºW)** é conhecida como **região Niño 3.4**. Esta região se localiza no Oceano Pacífico tropical e é comumente usada como uma métrica para determinar a fase da Oscilação Sul do El Niño (ENSO).


![Área dos Niños](https://www.ncei.noaa.gov/monitoring-content/teleconnections/nino-regions.gif)

ENSO é um padrão climático recorrente que envolve mudanças na SST no Oceano Pacífico tropical central e oriental, que possui duas fases alternadas:

- **El Niño:** a fase do ENSO caracterizada por SSTs mais quentes que a média no Oceano Pacífico tropical central e oriental, enfraquecimento dos ventos equatoriais de leste a oeste e aumento das chuvas no Pacífico tropical oriental.
- **La Niña:** a fase do ENSO que é caracterizada por SSTs mais frias que a média no Oceano Pacífico tropical central e oriental, ventos equatoriais de leste a oeste mais fortes e diminuição das chuvas no Pacífico tropical oriental.

## Seção 1.1: Processos Climáticos no Pacífico Tropical

Para entender melhor os processos do sistema climático que resultam nos eventos El Niño e La Niña, vamos primeiro considerar as condições climáticas típicas no Oceano Pacífico tropical. Os **ventos alísios** são ventos que sopram de leste a oeste ao norte e ao sul do equador (às vezes referidos como ventos "de leste" já que os ventos têm origem no leste e sopram em direção ao oeste). A razão pela qual os ventos alísios sopram de leste a oeste está relacionada à rotação da Terra, que faz com que os ventos no Hemisfério Norte se curvem para a direita e os ventos no Hemisfério Sul se curvem para a esquerda. Isso é conhecido como o **efeito de Coriolis**.

Se a rotação da Terra afeta o movimento do ar, você acha que também influencia o movimento da água do oceano superficial? Sim! Conforme os ventos alísios sopram sobre o Oceano Pacífico tropical, eles movem a água devido ao atrito na superfície do oceano. Mas devido ao efeito de Coriolis, a água da superfície do oceano se move para a direita da direção do vento no Hemisfério Norte e para a esquerda da direção do vento no Hemisfério Sul. No entanto, a velocidade e a direção do movimento da água mudam com a profundidade. A água da superfície do oceano se move em um ângulo em relação ao vento, e a água abaixo da superfície se move em um ângulo ligeiramente maior, e a água abaixo disso gira em um ângulo ainda maior. A direção média de toda essa água em movimento é aproximadamente um ângulo reto em relação à direção do vento. Esse valor médio é conhecido como **transporte de Ekman**. Como esse processo é impulsionado pelos ventos alísios, a intensidade desse transporte de água oceânica varia em resposta a mudanças na intensidade dos ventos alísios.

## Seção 1.2: Interações Oceano-Atmosfera Durante El Niño e La Niña

Então, como tudo isso se relaciona com El Niño e La Niña? Mudanças na intensidade dos ventos alísios do Oceano Pacífico e o impacto resultante no transporte de Ekman criam variações na SST do Pacífico tropical, o que resulta em mudanças nos padrões de circulação atmosférica e nas chuvas.

Durante um evento El Niño, **_os ventos alísios de leste são mais fracos_**. Como resultado, menos água superficial quente é transportada para o oeste via transporte de Ekman, o que causa um acúmulo de água superficial quente no Pacífico equatorial oriental. Isso cria SSTs mais quentes que a média no Oceano Pacífico equatorial oriental. A atmosfera responde a esse aquecimento com um aumento no movimento ascendente do ar e chuvas acima da média no Pacífico oriental. Em contraste, durante um evento La Niña, **_os ventos alísios de leste são mais fortes_**. Como resultado, mais água superficial quente é transportada para o oeste via transporte de Ekman, e água fria de mais profundidade no oceano sobe no Pacífico oriental durante um processo conhecido como ressurgência. Isso cria SSTs mais frias que a média no Oceano Pacífico equatorial oriental. Esse resfriamento diminui o movimento ascendente do ar no Pacífico oriental, resultando em condições mais secas que a média.

Neste tutorial, examinaremos as temperaturas de SST para explorar variações no sistema climático que ocorrem durante os eventos El Niño e La Niña. Especificamente, vamos plotar e interpretar dados de SST do OISST da região Niño 3.4.









# Seção 2: Calculando o índice ONI


### Fazendo o download dos dados, carregando os arquivos e primeiros plots

In [ ]:
# retrive (fetch) the SST data we are going to be working on
sst_path = DATASETS.fetch("CESM2_sst_data.nc")

# open the file we acquired with xarray
sst_data = xr.open_dataset(sst_path)

# remember that one degree spatial cell is not constant around the globe, each is a different size in square km.
# we need to account for this when taking averages for example

# fetch the weight for each grid cell
gridvars_path = DATASETS.fetch("CESM2_grid_variables.nc")

# open and save only the gridcell weights whose variable name is 'areacello'
# here the 'o' at the end refers to the area cells of the 'ocean' grid
areacello_data = xr.open_dataset(gridvars_path).areacello

# merge the SST and weights into one easy to use dataset - ds stands for dataset
ds_sst = xr.merge([sst_data, areacello_data])
ds_sst

In [ ]:
# define the plot size
fig = plt.figure(figsize=(12, 6))

# asssign axis and define the projection -  for a round plot
ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))

# add coastlines - this will issue a download warning but that is ok
ax.coastlines()

# add gridlines (lon and lat)
ax.gridlines()

# plots the first time index (0) of SST (variable name 'tos') at the first time
ds_sst.tos.isel(time=0).plot(
    ax=ax,
    transform=ccrs.PlateCarree(),  # give our axis a map projection
    vmin=-2,
    vmax=30,  # define the temp range of the colorbarfrom -2 to 30C
    cmap="coolwarm",  # choose a colormap
)

### Demo interativo

Você pode visualizar como serão os próximos momentos no modelo usando a barra deslizante interativa abaixo.

In [ ]:
# a bit more complicated code that allows interactive plots
# group the dataset by month
ds_sst_clim = ds_sst.tos.groupby("time.month")

# find the monthly climatology in the Nino 3.4 region
ds_sst_clim = ds_sst_clim.mean(dim="time")

gv.extension("bokeh")  # load Bokeh
dataset_plot = gv.Dataset(
    ds_sst_clim #.isel(time=slice(0, 10))
)  # slice only the first 10 timepoint, as it is a time consuming task
images = dataset_plot.to(gv.Image, ["longitude", "latitude"], "tos", "month")
images.opts(
    cmap="inferno",
    colorbar=True,
    width=600,
    height=400,
    projection=ccrs.Robinson(),
    clabel="Sea Surface Temperature [˚C]",
) * gf.coastline

## Seção 2.1: Gerando a TSM para o Niño 3.4

Você pode ter percebido que as coordenadas de longitude (`lon`) dos dados de SST estão organizadas entre 0°–360°E.

In [ ]:
ds_sst.lon

Isso difere da forma típica como usamos a `longitude` (-180°:180°). Como convertemos o valor da longitude entre esses dois sistemas (0°:360° versus -180°:180°)?

Vamos usar `lon2` para se referir ao sistema de longitude de `0°:360°`, enquanto `lon` se refere ao sistema de `-180°:180°`. `0°:360°` é equivalente a `0°:180°, -180°:0°`.

Em outras palavras, `lon2=181°` é o mesmo que `lon=-179°`. Portanto, no hemisfério ocidental, `lon2=lon+360`.

Portanto, a região Niño 3.4 deve ser (-5°:5°, 190°:240°) usando o sistema `lon2`.

Agora que identificamos os valores de longitude que precisamos selecionar, existem algumas maneiras de selecionar a região Niño 3.4. Vamos demonstrar como usar ambas abaixo.

1. `sel()`

In [ ]:
# select just the Nino 3.4 region (note our longitude values are in degrees east) by slicing
tos_nino34_op1 = ds_sst.sel(lat=slice(-5, 5), lon=slice(190, 240))
tos_nino34_op1

2. `where()`


In [ ]:
tos_nino34_op2 = ds_sst.where(
    (ds_sst.lat < 5) & (ds_sst.lat > -5) & (ds_sst.lon > 190) & (ds_sst.lon < 240),
    drop=True,
)  # use dataset where function. use boolean commands
tos_nino34_op2

Você pode verificar que `tos_nino34_op1` e `tos_nino34_op2` são iguais comparando os índices de latitude e longitude.

Precisamos apenas de uma dessas opções, então vamos escolher a segunda opção e atribuí-la à variável que usaremos daqui para frente.

In [ ]:
tos_nino34 = tos_nino34_op2

Vamos utilizar o mesmo código que usamos para plotar toda a Terra, mas desta vez focando apenas na fatia da região Niño 3.4.

In [ ]:
# define the figure size
fig = plt.figure(figsize=(12, 6))

# assign axis and projection
ax = plt.axes(projection=ccrs.Robinson(central_longitude=180))

# add coastlines
ax.coastlines()

# add gridlines (lon and lat)
ax.gridlines()

# plot as above
tos_nino34.tos.isel(time=0).plot(
    ax=ax, transform=ccrs.PlateCarree(), vmin=-2, vmax=30, cmap="coolwarm"
)

# make sure we see more areas of the earth and not only the square around Niño 3.4
ax.set_extent((120, 300, 10, -10))

## Seção 2.3: Calcular a Climatologia e Anomalias

Agora que selecionamos nossa área, podemos calcular a anomalia mensal primeiro agrupando todos os dados por mês e, em seguida, subtraindo a climatologia mensal de cada mês.

In [ ]:
# group the dataset by month
tos_nino34_mon = tos_nino34.tos.groupby("time.month")

# find the monthly climatology in the Nino 3.4 region
tos_nino34_clim = tos_nino34_mon.mean(dim="time")

# find the monthly anomaly in the Nino 3.4 region
tos_nino34_anom = tos_nino34_mon - tos_nino34_clim

# take the area weighted average of anomalies in the Nino 3.4 region
tos_nino34_anom_mean = tos_nino34_anom.weighted(tos_nino34.areacello).mean(
    dim=["lat", "lon"]
)
tos_nino34_anom_mean


## Seção 2.4: Suavizar a Série Temporal de Anomalias

A ENSO ocorre em escalas temporais interanuais (alguns anos ou mais). Para isolar a variabilidade desse fenômeno de longo prazo na região Niño 3.4, podemos suavizar as flutuações devido à variabilidade em escalas temporais mais curtas. Para isso, aplicaremos uma média móvel de 3 meses à nossa série temporal de anomalias de SST.

In [ ]:
oni_index = tos_nino34_anom_mean.rolling(time=3, center=True).mean()


In [ ]:
oni_serie = np.squeeze(oni_index)
tos_nino34_anom_mean = np.squeeze(tos_nino34_anom_mean)

a = oni_index.to_dataframe().reset_index()
dates = [datetime(int(a.time[x].strftime('%Y')), a.month[x], 1) for x in range(0,len(a))]

# define the plot size
fig = plt.figure(figsize=(12, 6))

# assign axis
ax = plt.axes()

# plot the monhtly data on the assigned axis
ax.plot(dates, tos_nino34_anom_mean, label='monthly', color="tomato")

# plot the smoothed data on the assigned axis
ax.plot(dates, oni_serie, label='3-month running mean', color="royalblue")

# add legend
plt.legend()

# add ylabel
ax.set_ylabel("Temprature Anomalies")

# add xlabel
ax.set_xlabel("Year")


# Seção 3: Identificar Eventos El Niño e La Niña

Vamos destacar valores acima de $\pm$0.5, correspondendo aproximadamente a eventos de El Niño (quente) e La Niña (frio).

In [ ]:
oni_Tai = np.squeeze(oni_index)

# time_tos = cftime.date2num(teste.time, 'days since 1970-01-01 00:00:00', calendar='gregorian')
time_tos = mdates.date2num(oni_Tai.time)

%matplotlib inline
# Adding labels and a grid
fig = plt.figure(figsize=(12,6))
ax = fig.add_subplot(1,1,1)

# Plot the series of data
ax.plot(time_tos, oni_Tai, label='Niño 3', color="black")
ax.set_xlabel('Time')
ax.set_ylabel('SST Anomaly (°C)')
ax.set_title('Oceanic Niño Index (ONI)')

ax.axhline(0, color="black", lw=0.5)  # add a black line at x=0
ax.axhline(
    0.5, color="black", linewidth=1.5, linestyle="dashed"
)  # add a black line at x=0.5
ax.axhline(
    -0.5, color="black", linewidth=1.5, linestyle="dashed"
)  # add a black line at x=-0.5
ax.grid(True)

plt.fill_between(  # plot with color in between
    time_tos,  # x values
    # top boundary - y values above 0.5
    oni_Tai.where(oni_Tai >= 0.5),
    0.5,  # bottom boundary - 0.5
    color="red",  # color
    alpha=0.9,  # transparency value_mean_mean_mean_mean_mean
)


plt.fill_between(  # plot with color in between
    time_tos,  # x values
    # top boundary - y values below 0.5
    oni_Tai.where(oni_Tai <= -0.5),
    -0.5,  # bottom boundary - -0.5
    color="blue",  # color
    alpha=0.9,  # transparency value_mean_mean_mean_mean_mean
)


# Adiciona formatação de data ao eixo x
ax.xaxis_date()
ax.set_ylim([-3, 3])

### Perguntas 3:

Agora que normalizamos os dados e destacamos as anomalias de SST que correspondem aos eventos de El Niño (quente) e La Niña (frio), considere as seguintes perguntas:

1. Com que frequência ocorrem eventos de El Niño e La Niña ao longo do período de tempo estudado aqui?
2. Quando ocorreram os eventos de El Niño e La Niña mais fortes durante este período de tempo?
3. Considerando as interações oceano-atmosfera que causam eventos de El Niño e La Niña, você pode formular hipóteses sobre possíveis razões pelas quais um evento de El Niño ou La Niña pode ser mais forte do que outros?

# Resumo

Neste tutorial, aprendemos a utilizar uma variedade de ferramentas do Xarray para examinar variações na SST durante eventos de El Niño e La Niña. Praticamos o carregamento de dados de SST do modelo CESM2 e a mascaragem de dados usando a função `.where()` para análises focadas. Também calculamos climatologias e anomalias usando a função `.groupby()`, e aprendemos a calcular médias móveis usando a função `.rolling()`. Por fim, calculamos, normalizamos e plotamos o Índice Oceânico Niño (ONI), aprimorando nossa compreensão da Oscilação Sul do El Niño (ENSO) e seus impactos nos padrões climáticos globais.